## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from pprint import pprint
import json

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Dingle,PH,10.9995,122.6711,79.77,81,100,10.47,overcast clouds
1,1,Labytnangi,RU,66.6572,66.4183,-24.57,100,92,2.64,overcast clouds
2,2,Kodiak,US,57.7900,-152.4072,38.71,87,100,5.75,mist
3,3,Kapaa,US,22.0752,-159.3190,79.12,67,0,3.00,clear sky
4,4,Saskylakh,RU,71.9167,114.0833,-28.21,100,35,1.74,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
# Code written with params as: min_temp= 75, max_temp= 95
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# "4a." has proven there are no null values, therefore the original DataFrame is still valid.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Dingle,PH,79.77,overcast clouds,10.9995,122.6711,
3,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
7,Bluff,NZ,76.80,light rain,-46.6000,168.3333,
9,Alofi,NU,76.89,light intensity drizzle,-19.0595,-169.9187,
14,Rikitea,PF,78.35,broken clouds,-23.1203,-134.9692,
17,Iralaya,HN,75.49,clear sky,15.0000,-83.2333,
18,Saint-Philippe,RE,80.15,moderate rain,-21.3585,55.7679,
24,Bagotville,AU,86.02,overcast clouds,-28.9833,153.4167,
25,Port Alfred,ZA,77.63,overcast clouds,-33.5906,26.8910,
32,Arraial Do Cabo,BR,76.19,moderate rain,-22.9661,-42.0278,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng= row["Lng"]

    params["location"]= f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels= requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found...skipping.")

hotel_df
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Dingle,PH,79.77,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
3,Kapaa,US,79.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Bluff,NZ,76.80,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
9,Alofi,NU,76.89,light intensity drizzle,-19.0595,-169.9187,Taloa Heights
14,Rikitea,PF,78.35,broken clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
650,Kismayo,SO,78.24,overcast clouds,-0.3582,42.5454,Agan-Hotel
651,Makasar,ID,80.65,few clouds,-5.1400,119.4221,Hotel Horison Ultima Makassar
654,Paracuru,BR,82.42,few clouds,-3.4100,-39.0306,Hotel Vento Brasil
657,Namuac,PH,76.93,few clouds,18.6109,121.1657,Mlin Hotel


In [9]:
hotel_df.count()

City           160
Country        160
Max Temp       160
Description    160
Lat            160
Lng            160
Hotel Name     160
dtype: int64

In [10]:
#determine how many rows have no hotel name
(hotel_df["Hotel Name"].values == '').sum()

6

In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value= float("NaN")
hotel_df.replace('', nan_value, inplace=True)
hotel_df.dropna(subset= ["Hotel Name"], inplace=True)

(hotel_df["Hotel Name"].values == '').sum()

0

In [12]:
#confirmation empty rows dropped
hotel_df.count()

City           154
Country        154
Max Temp       154
Description    154
Lat            154
Lng            154
Hotel Name     154
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file= "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))